<a href="https://colab.research.google.com/github/loris2222/ChocoData/blob/main/code/ChocoData_from_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChocoData

**Chocolate** is the food we all know and love. It dates back to the Olmec culture (19th-11th century BCE), and it has been consumed in some form by Maya and Aztecs too. Chocolate is consumed today more than ever and we advocate for consumer awareness concerning how chocolate is made. To this end, we explore some chocolate bar rating data, contained in the [Chocolate Bar Ratings](https://www.kaggle.com/rtatman/chocolate-bar-ratings) Kaggle dataset. The idea is to present, analyze, and compare the best chocolate bars throughout a visual data-analysis tour.

The `ChocoData` notebook is the exam project of the Ph.D. course **Data and Results Visualization**, held by Prof. Daniele Loiacono at Politecnico di Milano in December 2021. The authors are Loris Giulivi, Paolo Boffi, and Alberto Archetti. It is provided under the MIT license.

![testo del link](https://www.chocolate7.com/wp-content/uploads/2020/07/scopri_il_tuo_cioccolato_home.jpg)


In [15]:
# Set COLORBLIND to True to enable colorblind mode
COLORBLIND = True

## Set up the stage

In this section, we import the main plotting libraries, `seaborn` and `matplotlib` and prepare the dataset for further processing. We fix duplicate, wrong, and missing values. Finally, we define some helper functions that will be useful later.

Note: you must restart the runtime after upgrading `matplotlib`!

### Imports

In [16]:
# Update matplotlib to the last available version; without this ax.bar_label 
# does not work (used to display bar values on seaborn barplots); every time 
# this line of code is executed for the first time, we need to restart the 
# runtime
! pip install matplotlib --upgrade matplotlib

In [20]:
import os
from os import path
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import geopandas
import folium
import csv
import math
import ipywidgets as widgets
from IPython.display import display
from shapely.geometry import shape

sns.set_theme(style="darkgrid")

# All the plots will be saved in the ./images folder
if not os.path.exists('images'):
        os.mkdir('images')

AttributeError: module 'numpy.random' has no attribute 'mtrand'

### Download the dataset

In [ ]:
# Download and extract the dataset; the flavors_of_cacao.csv file is now 
# available locally
! wget -nc https://www.dropbox.com/sh/duqeg9dx2o8s18r/AAAX_rCz1dwN8rs1LMwH1Jg1a
! mv AAAX_rCz1dwN8rs1LMwH1Jg1a flavors_of_cacao.zip
! unzip -n flavors_of_cacao.zip && rm -f flavors_of_cacao.zip

### Helper functions

In [ ]:
def get_palette(n, ordered=False, invert=False, colorblind=COLORBLIND):
    """ Returns a color palette on n elements for Seaborn."""
    palette = sns.color_palette('deep', n)
    if colorblind:
        palette = sns.color_palette('colorblind', n)
    if ordered:
        palette = sns.color_palette('dark:#69d', n)[::-1]
    if invert:
        palette = palette[::-1]
    return palette

def annotate_with_counts(ax):
    """Adds counts on top of a Seaborn countplot."""
    for p in ax.patches: 
        ax.annotate('{:d}'.format(p.get_height()), 
                    (p.get_x() + p.get_width() * 0.5, p.get_height() + 5), 
                    ha = 'center')
    return ax

def rename_axes(ax, xlabel, ylabel):
    """"Renames the axes of a plot."""
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    return ax

def vocabularize(vocab_dict, force=False):
    """Returns a vocabularize function for Pandas `apply`."""
    def vocabularize_with_dict(x):
        for b in vocab_dict.keys():
            if x.find(b) != -1:
                return vocab_dict[b]
        if force:
            return 'Unknown'
        return x
    return vocabularize_with_dict

def html_country(country):
    dimx=0
    dimy=0
    infofile = open(os.path.join(os.getcwd(), "./images/best_companies_by_country/iminfo.txt"))
    csv_file = csv.reader(infofile, delimiter=",")
    for row in csv_file:
        if row[0] == country:
            dimx = int(row[1])/4
            dimy = int(row[2])/4
            break
    infofile.close()
    if dimy==0:
        dim = 200
    html = "<img src='http://127.0.0.1:8887/images/best_companies_by_country/"+country+".png' alt='chart' style='height:"+str(math.floor(dimy))+"px'>"

    return html

### Data loading

In [ ]:
# Load the dataset
chocodata = pd.read_csv("flavors_of_cacao.csv")

# Rename columns
columns = ['company', 'bar_name', 'ref', 'review_date', 'cocoa_percent',
           'company_location', 'rating', 'bean_type', 'bean_origin']
chocodata = chocodata.rename(columns=dict(zip(chocodata.columns, columns)))

# Reorder columns
columns = ['bar_name', 'company', 'company_location', 'rating', 'ref', 
           'review_date', 'cocoa_percent', 'bean_type', 'bean_origin']
chocodata = chocodata[columns]

# Define data types
chocodata['cocoa_percent'] = chocodata['cocoa_percent'].apply(
    lambda x: float(str(x).strip('%')) * 0.01)
chocodata = chocodata.astype({
    'bar_name': str,
    'company': str, 
    'company_location': str, 
    'rating': float, 
    'ref': int, 
    'review_date': int,
    'cocoa_percent': float, 
    'bean_type': str, 
    'bean_origin': str
})

# Sort values by insert order in the dataset
chocodata = chocodata.sort_values(by=['ref', 'review_date', 'company'])

# Print the first 5 rows
chocodata.head()

### Data cleaning

1. We fix inconsistencies in country names,
2. we modify the `bean_type` column in order to limit its values to `Criollo`, `Forastero`, and `Trinitario`,
3. we convert any missing value to `Unknown`.

In [ ]:
# Fix country names
country_dict = {
    'Amsterdam': 'Netherlands',
    'Eucador': 'Ecuador',
    'Niacragua': 'Nicaragua',
    'U.S.A.': 'United States of America',
    'Trinidad': 'Trinidad and Tobago',
    'Congo': 'Dem. Rep. Congo',
    'Dominican Republic': 'Dominican Rep.',
    'Domincan Republic': 'Dominican Rep.',
    'Solomon Islands': 'Solomon Is.'
}
chocodata['company_location'] = chocodata['company_location'].apply(
    vocabularize(country_dict))

# Fix bean types
bean_dict = {
    'Criollo': 'Criollo',
    'Forastero': 'Forastero',
    'Trinitario': 'Trinitario'
}
chocodata['bean_type'] = chocodata['bean_type'].apply(
    vocabularize(bean_dict, force=True))

# Fix bean origins
prod_dict = {
    'Trinidad': 'Trinidad and Tobago',
    'Trinidad, Tobago': 'Trinidad and Tobago',
    'Trinidad-Tobago': 'Trinidad and Tobago',
    'Tobago': 'Trinidad and Tobago',
    'Carribean': 'Caribbean',
    'Ghana': 'Ghana',
    'Venezuela': 'Venezuela',
    'Ven.': 'Venezuela',
    'Ven': 'Venezuela',
    'Venez': 'Venezuela',
    'Dominican Republic': 'Dominican Rep.',
    'Domincan Republic': 'Dominican Rep.',
    'Colombia': 'Colombia',
    'Ecuador': 'Ecuador',
    'Peru': 'Peru',
    'Peru(SMartin,Pangoa,nacional)': 'Peru',
    'South America': 'South America',
    'Principe': 'Sao Tome and Principe',
    'Haiti': 'Haiti',
    'Mad.': 'Madagascar',
    'Congo': 'Dem. Rep. Congo',
    'Amsterdam': 'Netherlands',
    'Solomon Islands': 'Solomon Is.'
}
chocodata['bean_origin'] = chocodata['bean_origin'].apply(
    vocabularize(prod_dict))

# Un-capitalize country names to be compliant with naming convention
chocodata['bean_origin'] = chocodata['bean_origin'].str.lower()
chocodata['company_location'] = chocodata['company_location'].str.lower()

# Substitute all the empty and NaN cells with 'Unknown'
for c in columns:
    chocodata[c] = chocodata[c].replace(r'^\s+$', 'Unknown', regex=True)
    chocodata[c] = chocodata[c].fillna('Unknown')

chocodata.head()

## Summary statistics

In this section, we describe the attributes of the dataset and provide an overall insight about its content using summary statistics.

In [ ]:
# Define column descriptions
column_descriptions = {
    'bar_name': 'The bar name or the specific geo-region for the bar', 
    'company': 'Name of the company manufacturing the bar', 
    'company_location': 'Manufacturer base country', 
    'rating': 'Expert rating for the bar', 
    'ref': 'Review creation value. Higher = more recent', 
    'review_date': 'Date of publication of the review',
    'cocoa_percent': 'Cocoa percentage (darkness)', 
    'bean_type': 'The variety (breed) of bean used, if provided', 
    'bean_origin': 'The broad geo-region of origin for the bean'
}

# Count unique values of each column
chocodata_uniques = {}
for c in columns:
    chocodata_uniques[c] = chocodata[c].unique()

# Define a dictionary of columns paired with unique values and attribute 
# descriptions
unique_values = {'column': ['total_samples'], 
                 'unique_values': [chocodata.shape[0]],
                 'type': [type(chocodata.shape[0])],
                 'description': ['Total number of samples in the dataset']}
for c in columns:
    unique_values['column'].append(c)
    unique_values['unique_values'].append(len(chocodata_uniques[c]))
    unique_values['type'].append(type(chocodata[c][0]))
    unique_values['description'].append(column_descriptions[c])

# Convert dictionary to DataFrame
unique_values = pd.DataFrame(unique_values)
unique_values

In [ ]:
# Print summary statistics of numerical attributes
chocodata.describe()

## Ratings timeline

In this section, we plot the ratings trend throughout the years, highlighting the rating counts.


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 10))
fig.suptitle('Average bar ratings per year', y=0.94)

palette = get_palette(len(chocodata['review_date'].unique()), ordered=True)
sns.countplot(ax=ax[0], data=chocodata, x='review_date', palette=palette)
annotate_with_counts(ax[0])
rename_axes(ax[0], 'Year', 'Count')

sns.boxplot(ax=ax[1], data=chocodata, x='review_date', y='rating', 
            palette=palette)
rename_axes(ax[1], 'Year', 'Rating')

plt.savefig('images/ratings_timeline.pdf', bbox_inches='tight')

## Companies

In this section, we collect a set of visualizations concerning company ratings.


### Map of companies in the dataset

In [ ]:
# Import world map from geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world['name'] = world['name'].str.lower()

# Fix inconsistent name for ivory coast which uses non-ascii character
world.loc[(world['name'] == "côte d'ivoire"), 'name'] = "ivory coast"

# Merge company locations with world data and count companies per country
merge = chocodata.merge(world, how='left', left_on=["company_location"], right_on=["name"])
company_location_counts = merge.groupby(["name"]).size().reset_index(name='counts')

# Initialize choropleth map
map = folium.Map()
folium.Choropleth(
    geo_data=world,
    name="chocochoco",
    data=company_location_counts,
    columns=["name", "counts"],
    key_on="feature.properties.name",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Chocolate making companies",
).add_to(map)

# Show map
display(map)

### Companies per number of reviews


In [ ]:
# Get a list of all the unique values of the company column
companies = chocodata_uniques['company']

# Get a list of all the unique values of the company_location column
countries = chocodata_uniques['company_location']

# Create a dictionary of the companies and their average evaluations
d = {'company': [],
     'avg_rating': [],
     'std_dev': [],
     'num_revs': []}

# For each company save its average evaluation in a dataframe
for c in companies:
    company_data = chocodata[chocodata.company == c]
    d['company'].append(c)
    d['avg_rating'].append(company_data['rating'].mean())
    d['std_dev'].append(company_data['rating'].std())
    d['num_revs'].append(company_data['ref'].count())

avg_companies = pd.DataFrame(d)

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Companies per number of reviews')
ax.set_ylim((0, 140))
palette = get_palette(len(avg_companies['num_revs'].unique()), ordered=True)
sns.countplot(ax=ax, data=avg_companies, x='num_revs', palette=palette)
annotate_with_counts(ax)
rename_axes(ax, 'Reviews count', 'Companies count')

plt.savefig('images/companies_per_number_of_reviews.pdf', bbox_inches='tight')

### Top-rated companies

In [ ]:
# Drop all companies with less than N reviews
MIN_NUM_OF_REVIEWS = 5

# Plot the top-N companies
TOP_COMPANIES = 10

# Remove all the entries with too few bars
avg_companies = avg_companies.drop(
    avg_companies[avg_companies.num_revs < MIN_NUM_OF_REVIEWS].index)

# Extract the ten better companies in term of average chocolate rating
best_companies = avg_companies.nlargest(TOP_COMPANIES, 
                                        columns=['avg_rating'])

# Plot them through a horizontal barplot
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
fig.suptitle('Top-{} companies'.format(TOP_COMPANIES))
palette = get_palette(len(best_companies), ordered=True, invert=True)
ax = sns.barplot(data=best_companies, x='avg_rating', y='company', 
                 palette=palette)
rename_axes(ax, 'Average rating', 'Company')
ax.bar_label(ax.containers[0], fmt='%.2f')
ax.set_xlim((3, 4))
sns.despine()

plt.savefig('images/top{}_companies.pdf'.format(TOP_COMPANIES), 
            bbox_inches='tight')

### Best company per country

In [ ]:
# Collect the best company for each country
t = {'country': [],
     'best_company': [],
     'avg_rating': []}

# For each country save its companies
for c in countries:
    companies_country = chocodata.loc[chocodata['company_location'] == c]
    # Create a dataframe in which to save the companies and their average evaluations
    d = {'company': [],
         'avg_rating': []}
    # For each company in country c, compute the average rating of its bars
    for comp in companies_country.company:
        # Compute the average rating for company comp of country c
        company_data = companies_country[companies_country.company == comp]
        d['company'].append(company_data['company'].values[0])
        d['avg_rating'].append(company_data['rating'].mean())
    avg_companies = pd.DataFrame(d)
    # Extract the best company for country c
    best_company = avg_companies.nlargest(1, ['avg_rating'])
    # Insert the company in the dataframe
    t['country'].append(c)
    t['best_company'].append(best_company['company'].values[0])
    t['avg_rating'].append(best_company['avg_rating'].values[0])

loc_companies = pd.DataFrame(t)
loc_companies = loc_companies.sort_values('country')

loc_companies

### Map of companies per country and best companies

In [ ]:
# Changes matplotlib backend to prevent outputting all the figures
%matplotlib agg
%matplotlib agg

# Create a dataframe in which to save the best companies for each country
d = {'country': [],
     'best_company': [],
     'avg_rating': []}
loc_companies = pd.DataFrame(d)

# Create a dataframe to save the average rating of all bars of a country
d = {'country': [],
     'avg_rating': []}
country_avg_ratings = pd.DataFrame(d)

# Foreach country save its best companies
lines = []
txtfile = open(os.path.join(os.getcwd(), "./images/best_companies_by_country/iminfo.txt"), 'w')
for c in countries:
    companies_country = chocodata.loc[chocodata['company_location'] == c]
    mean_country_rating = companies_country['rating'].mean()
    d = {'country': [c], 'avg_rating': [mean_country_rating]}
    avg_country = pd.DataFrame(d)
    tmp = [country_avg_ratings, avg_country]
    country_avg_ratings = pd.concat(tmp)

    # Create a dataframe in which to save the companies and their average evaluations
    d = {'company': [], 'avg_rating': []}
    avg_companies = pd.DataFrame(d)

    # Foreach company in country c, compute the average rating of its bars
    for comp in pd.unique(companies_country.company):
        company = companies_country[companies_country.company == comp]
        mean_rating = company['rating'].mean()

        # Insert that company and its average rating inside the avg_companies dataframe
        d = pd.DataFrame(
            {'company': [comp],
             'avg_rating': [mean_rating]})
        tmp = [avg_companies, d]
        avg_companies = pd.concat(tmp)

    # Extract the top three companies for country c
    n_companies = 3
    best_company = avg_companies.nlargest(n_companies, ['avg_rating'])

    # Insert the company in the dataframe
    actual_n_companies = np.shape(best_company['company'].values)[0]
    country_rep_list = [c]*actual_n_companies
    company_list = best_company['company'].values
    rating_list = best_company['avg_rating'].values
    d = pd.DataFrame(
        {'country': country_rep_list,
         'best_company': company_list,
         'avg_rating': rating_list})
    tmp = [loc_companies, d]
    loc_companies = pd.concat(tmp, ignore_index=True)

    # Create a figure for each country to be used as a tooltip in the map
    sns.set(font_scale = 4)
    _, axes = plt.subplots(actual_n_companies, 1, figsize=(5,5*actual_n_companies))
    for i in range(actual_n_companies):
        company = companies_country[companies_country.company == company_list[i]]
        ax = sns.boxplot(ax=(axes[i] if actual_n_companies>1 else axes),y=company['company'], x=company['rating'], linewidth=2.5)
        ax.set(ylabel=None)

    # Save the figures in the images folder
    tosave = ax.get_figure()
    tosave.savefig(os.path.join(os.getcwd(), "./images/best_companies_by_country/"+country_rep_list[0]+".png"), bbox_inches = 'tight')
    im = Image.open(os.path.join(os.getcwd(), "./images/best_companies_by_country/"+country_rep_list[0]+".png"))
    lines.append(country_rep_list[0]+","+str(im.size[0])+","+str(im.size[1])+"\n")

# Save the helper file that will be used to build the html
txtfile.writelines(lines)
txtfile.close()
loc_companies.set_index('country')
loc_companies.sort_values('country')
sns.set(font_scale = 1)
%matplotlib inline

# Merge world data with the best companies data
merge = country_avg_ratings.merge(world, how='left', left_on=["country"], right_on=["name"])

# Initialize choropleth map
map = folium.Map(location=[48, -102], zoom_start=1)
cp = folium.Choropleth(
    geo_data=world,
    name="chocochoco",
    data=merge,
    columns=["name", "avg_rating"],
    key_on="feature.properties.name",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Mean chocolate ratings by country and best companies",
).add_to(map)

# Add popups to the map
for s in cp.geojson.data['features']:
    s['properties']['popup_html'] = html_country(s['properties']['name'])

folium.GeoJsonPopup(fields=['popup_html'], labels=False).add_to(cp.geojson)
folium.LayerControl().add_to(map)

# Show map
display(map)

## Cocoa beans

In this section, we collect a set of visualizations concerning cocoa beans ratings and cocoa percentage ratings. 

### Bean types distribution

In [ ]:
# Generate a new dataframe containing bean counts
chocodata_beans = chocodata.sort_values(by='bean_type')
bean_counts = chocodata_beans.groupby('bean_type').size().reset_index(
    name='counts')

fig, ax = plt.subplots(1, 1, figsize=[8, 10])
plt.pie(x=bean_counts['counts'], 
        autopct="%.1f%%", 
        labels=bean_counts['bean_type'],
        colors=get_palette(4))
fig.suptitle("Bean types distribution", y=0.85)

plt.savefig('images/bean_types_distribution.pdf', bbox_inches='tight')

### Ratings per bean type


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
fig.suptitle('Ratings per bean type')
sns.boxplot(ax=ax, 
            data=chocodata_beans, 
            x='bean_type', 
            y='rating', 
            palette=get_palette(4))
rename_axes(ax, 'Bean type', 'Rating')

plt.savefig('images/ratings_per_bean_type.pdf', bbox_inches='tight')

### Ratings per cocoa percent

In [ ]:
# Control the aggregation precision
STEP = 0.05
assert STEP >= 0.05, "Aggregation too thin!"

# Define a new dataframe with aggregated percentages
percentages = np.arange(0.4, 1.0 + STEP * 0.5, STEP)
d = {'perc': [], 'rating': []}
for bar in chocodata.iterrows():
    bar = bar[1]
    for p in percentages:
        if p <= bar['cocoa_percent'] < p + STEP:
            d['perc'].append(p)
            d['rating'].append(bar['rating'])
perc_ratings = pd.DataFrame(d)


fig, ax = plt.subplots(2, 1, figsize=(8, 10))
fig.suptitle('Ratings per cocoa percent', y=0.94)
palette = get_palette(len(percentages), ordered=True)

sns.countplot(ax=ax[0], data=perc_ratings, x='perc', palette=palette)
annotate_with_counts(ax[0])
rename_axes(ax[0], 'Cocoa percent', 'Number of reviews')
ax[0].set_xticklabels([str(i) + '%' for i in range(40, 101, int(STEP * 100))])
ax[0].set_ylim((0, 1000))

sns.pointplot(ax=ax[1], 
              data=perc_ratings,
              x='perc', 
              y='rating',
              palette=palette)
ax[1].set_ylim((1, 5))
ax[1].set_xticklabels([str(i) + '%' for i in range(40, 101, int(STEP * 100))])
rename_axes(ax[1], 'Cocoa percent', 'Rating')

plt.savefig('images/ratings_per_cocoa_percent.pdf', bbox_inches='tight')

### Bean type and cocoa percent

In [ ]:
hue_order = ['Criollo', 'Forastero', 'Trinitario']
g = sns.jointplot(data=chocodata[chocodata['bean_type'] != 'Unknown'], 
                  x='cocoa_percent', 
                  y='rating', 
                  hue='bean_type', 
                  hue_order=hue_order,
                  palette=get_palette(3),
                  height=10)
g.ax_joint.set_xlim([0.4, 1.01])
g.ax_joint.set_ylim([0.93, 5.05])
g.ax_joint.legend(loc='upper left')
g.ax_joint.set_xticklabels([str(i) + '%' for i in range(40, 101, 10)])
g.fig.suptitle('Cocoa percent per bar rating', y=1.04)
rename_axes(g.ax_joint, 'Cocoa percent', 'Rating')

plt.savefig('images/bean_type_and_cocoa_percent.pdf', bbox_inches='tight')

### Top-rated bean exporters

In [ ]:
# Drop all countries with less than N reviews
MIN_NUM_OF_REVIEWS = 10

# Plot the top-N countries
TOP_COUNTRIES = 10

# Get a list of all the unique cocoa bean origins
b_origins = chocodata_uniques['bean_origin']

# Create the dictionary with the average grades with respect to the bean origin
d = {'bean_origin': [],
      'avg_rating': [],
      'num_revs':[]}

# Forveach bean type, compute its average evaluation
for o in b_origins:
    origin_data = chocodata[chocodata.bean_origin == o]
    d['bean_origin'].append(o)
    d['avg_rating'].append(origin_data['rating'].mean())
    d['num_revs'].append(origin_data['ref'].count())

# Convert the dictionary into a DataFrame
avg_bean_origin = pd.DataFrame(d)

# Remove all the grades linked to an unknown bean type
avg_bean_origin = avg_bean_origin[avg_bean_origin.bean_origin != 'Unknown']

# Remove all the entries with too few bars
avg_bean_origin = avg_bean_origin.drop(
    avg_bean_origin[avg_bean_origin.num_revs < MIN_NUM_OF_REVIEWS].index)

# Extract the 10 better bean types, in terms of average rating
best_countries = avg_bean_origin.nlargest(TOP_COUNTRIES, columns=['avg_rating'])

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
fig.suptitle('Top-{} bean exporters'.format(TOP_COUNTRIES))
palette = get_palette(len(best_countries), ordered=True, invert=True)
ax = sns.barplot(data=best_countries, 
                 x='avg_rating', 
                 y='bean_origin', 
                 palette=palette)
rename_axes(ax, 'Average rating', 'Bean origin')
ax.bar_label(ax.containers[0], fmt='%.2f')
ax.set_xlim((3, 3.6))
sns.despine()

plt.savefig('images/top{}_bean_exporters.pdf'.format(TOP_COUNTRIES), 
            bbox_inches='tight')

# TODO: check

In [ ]:
# Get a list of all the unique cocoa bean types
beans = chocodata_uniques['bean_type']

# Create the dictionary that saves the average grade with respect to the 
# cocoa bean type
d = {'bean_type': [],
     'avg_rating': [],
     'num_bars': [],
     'std_err': []}

for b in beans:
    beans_data = chocodata[chocodata.bean_type == b]
    d['bean_type'].append(b)
    d['avg_rating'].append(beans_data['rating'].mean())
    d['num_bars'].append(beans_data['ref'].count())
    d['std_err'].append(beans_data['rating'].std)

# convert the dictionary into a DataFrame
avg_bean_type = pd.DataFrame(d)
avg_bean_type = avg_bean_type[avg_bean_type['bean_type'] != 'Unknown']

# Order the DataFrame by the average rating
avg_bean_type = avg_bean_type.sort_values(by='avg_rating', ascending=False)

# Compute the minimum and the maximum rating in the DataFrame
min_x = avg_bean_type['avg_rating'].min()
max_x = avg_bean_type['avg_rating'].max()

# Plot them through a horizontal barplot
ax = sns.barplot(data=avg_bean_type, x='avg_rating', y='bean_type',
                palette=get_palette(3))
ax.set(xlabel='Average Bar Rating', ylabel='Bean Type')
ax.bar_label(ax.containers[0], fmt='%.2f')
ax.set(xlim=(min_x-0.1, max_x))
sns.despine()

plt.suptitle('Bran types scoreboard')
#plt.savefig('images/best_beanTypes.pdf', bbox_inches='tight')